## Self Attention Model
\begin{align}
Q &= X W^Q, \\
K &= X W^K, \\
V &= X W^V, \\
\text{Attention}(Q, K, V) 
  &= \operatorname{Softmax}\!\left(\frac{QK^\top}{\sqrt{d_k}}\right)V.
\end{align}

\begin{align}
\text{MultiHead}(X) 
  &= \text{Concat}\big(\text{head}_1, \ldots, \text{head}_h\big) W^O, \\
\text{head}_i 
  &= \operatorname{Attention}(X W_i^Q, X W_i^K, X W_i^V).
\end{align}


In [1]:
import torch
import torch.nn as nn
class myselfattention(nn.Module):
    def __init__(self, d_in, d_out,context_length,bias=False):
        super().__init__()
        self.W_value=nn.Linear(d_in,d_out,bias=bias)
        self.W_query=nn.Linear(d_in,d_out,bias=bias)
        self.W_key=nn.Linear(d_in,d_out,bias=bias)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New
    def forward(self, x): 
        _,num_tokens,_=x.shape
        value=self.W_value(x)
        query=self.W_query(x)
        key=self.W_key(x)
        attn_scores=key@query.mT
        attn_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)      
        return nn.Softmax(dim=-1)(attn_scores/key.shape[-1]**0.5)@value
myattention=myselfattention(4,4,3)
input=torch.rand(2,3,4)
output=myattention(input)
print(f"output size is: {output.shape}")

output size is: torch.Size([2, 3, 4])


In [2]:
class myselfattention_mh(nn.Module):
    def __init__(self, d_in, d_head, d_out,context_length,bias=False):
        super().__init__()
        self.heads=nn.ModuleList([myselfattention(d_in, d_head, context_length) for _ in range(d_out//d_head)])
        self.W_out=nn.Linear(d_out,d_out,bias)
    def forward(self, x): 
        out=torch.cat([self.heads[i](x) for i in range(len(self.heads))], dim=-1)
        return self.W_out(out)

In [3]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(myselfattention_mh(16,4,16,3))

1024

$\text{GELU}(x) \approx 0.5 \cdot x \cdot \left(1 + \tanh\left[\sqrt{\frac{2}{\pi}} \cdot \left(x + 0.044715 \cdot x^3\right)\right]\right)$

In [4]:
class mySilu(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x): 
        ## TODO:

$FFN(X)=silu(XW_1)W_2$ where $W_1$ size is $d\times 4d$, $W_2$ size is $4d\times d$

In [5]:
class ffn(nn.Module):
    def __init__(self,dim,dropout=0,bias=False):
        super().__init__()
        ## TODO
    def forward(self, x): 
        ## TODO

In [6]:
class myTransformer(nn.Module):
    def __init__(self, heads, dropout, hidden,context_length,bias=False):
        super().__init__()
        ## TODO
    def forward(self, x): 
        ## TODO
        return x

In [7]:
cfg={
    "layers": 12,
    "heads": 12,
    "dropout": 0.1,
    "context_length": 1024,
    "hidden_dim": 768,
    "voc_size": 50257
}
class myGPT2(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers=cfg["layers"]
        self.heads=cfg["heads"]
        self.dropout=cfg["dropout"]
        self.context_length=cfg["context_length"]
        self.hidden_dim=cfg["hidden_dim"]
        self.voc_size=cfg["voc_size"]
        self.wte=nn.Embedding(self.voc_size,self.hidden_dim)
        self.wpe=nn.Embedding(self.context_length,self.hidden_dim)
        self.layernorm=nn.LayerNorm(self.hidden_dim)
        self.finalLinear=nn.Linear(self.hidden_dim,self.voc_size,bias=False)
        self.transformerBlocks=nn.ModuleList([myTransformer(self.heads,self.dropout,self.hidden_dim,self.context_length) for _ in range(self.layers)])
    def forward(self, x): 
        batch,length=x.shape
        te=self.wte(x);
        pe=self.wpe(torch.arange(length));
        ## TODO
        return x

In [8]:
x=torch.tensor([[1, 2, 3], [4, 5, 6]])
model=myGPT2(cfg)
model(x).shape

torch.Size([2, 3, 50257])

In [9]:
input_text1="I study at Clemson"
input_text2="I am a student"
input=[input_text1,input_text2]
import tiktoken

# Load GPT-2 tokenizer
enc = tiktoken.get_encoding("gpt2")

#text = "Hello, I live in Atlanta."
tokens = [enc.encode(input_text) for input_text in input]


In [10]:
tokens

[[40, 2050, 379, 27801], [40, 716, 257, 3710]]

In [11]:
model(torch.tensor(tokens)).shape

torch.Size([2, 4, 50257])

In [12]:
def generate(model,tokens,max_length):
    for i in range(max_length):
        with torch.no_grad():
            x=model(tokens)
        x_pred=x[:,-1,:]
        _,max_ind=torch.max(x_pred, -1, keepdim=True)
        tokens=torch.cat([tokens, max_ind], dim=1)
    return tokens
generated=generate(model,torch.tensor(tokens),6)
#print(generate(model,torch.tensor(tokens),6))

In [13]:
generated

tensor([[   40,  2050,   379, 27801, 44093, 11191, 40749,  4096, 39619, 36195],
        [   40,   716,   257,  3710, 26114, 27351, 21809,  4048,  2358, 42344]])

In [14]:
for i, seq in enumerate(generated.tolist()):
    text = enc.decode(seq)
    print(f"Sequence {i}: {text}")

Sequence 0: I study at Clemson899 surrounded Seek basic Serbianlif
Sequence 1: I am a studentPad Tetwire female manag circuitry


In [15]:
enc.encode(text)

[40, 716, 257, 3710, 26114, 27351, 21809, 4048, 2358, 42344]